In [1]:
#importo las librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

In [2]:
#importo las librearias de Scikit Learn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor as KN
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.model_selection import GridSearchCV

In [3]:
# importo el dataset de Airbnb London, lo guardamos en el dataframe "london".
london = pd.read_csv(r'clusterai_regresion_dataset_airbnb_london.csv', delimiter=',', parse_dates = True)

In [4]:
# observo la cantidad de renglones y columnas del dataset importado
np.shape(london)

(75213, 16)

In [5]:
# exploro las columnas que tiene el dataframe importado
london.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [6]:
#visualizo los primeros 3 renglones de mi dataset
london.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9554,"Cozy, 3 minutes to Piccadilly Line",31655,Guy,NaN,Haringey,51.587767,-0.105666,Private room,35,1,131,2018-08-03,1.71,4,262
1,11076,The Sanctuary,40471,Rosa,NaN,Ealing,51.515645,-0.314508,Private room,70,2,2,2016-11-23,0.07,6,62
2,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.568017,-0.111208,Private room,45,1,14,2018-06-17,0.14,2,364


In [7]:
# cuento cuantos NaNs tengo por columna
london.isnull().sum()

id                                    0
name                                 31
host_id                               0
host_name                            13
neighbourhood_group               75213
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       20353
reviews_per_month                 20357
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [8]:
# elimino las rows que tienen NaNs en la columna "reviews_per_month"
london = london.dropna(subset = ["reviews_per_month"])

In [9]:
# reviso como queda mi dataframe luego de eliminar los renglones con NaNs en review per month
london.shape

(54856, 16)

In [10]:
#elimino las columnas que creo que no son de mi interes
london = london.drop(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'last_review', 'latitude', 'longitude'], axis=1)

In [11]:
london.head(3)

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Haringey,Private room,35,1,131,1.71,4,262
1,Ealing,Private room,70,2,2,0.07,6,62
2,Islington,Private room,45,1,14,0.14,2,364


In [12]:
london.isnull().sum()

neighbourhood                     0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [13]:
#verifico los valores maximos de las columnas del dataframe
np.max(london)

neighbourhood                     Westminster
room_type                         Shared room
price                                   10000
minimum_nights                           1000
number_of_reviews                         536
reviews_per_month                       15.56
calculated_host_listings_count           1034
availability_365                          365
dtype: object

In [14]:
#Obtengo los percentiles 0.97 de de "precio" y "minimun nights" para filtrar los outliers
price_q97 = london.price.quantile(0.975)
print("el cuantil 0.97 de la feature 'price' es = " + str(price_q97))
min_nights_q97 = london.minimum_nights.quantile(0.975)
print("el cuantil 0.97 de la feature 'minimum_nights' es = " + str(min_nights_q97))

el cuantil 0.97 de la feature 'price' es = 300.0
el cuantil 0.97 de la feature 'minimum_nights' es = 14.0


In [15]:
#Filtro mi dataset por los percentiles calculados y conservo los valores que ucmplen las dos condiciones al mismo tiempo
london_filt = london.loc[(london.price < price_q97) & (london.minimum_nights < min_nights_q97)]

In [16]:
#verifico como quedo mi nuevo dataframe filtrado
np.shape(london_filt)

(51827, 8)

In [17]:
london_filt.head(3)

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Haringey,Private room,35,1,131,1.71,4,262
1,Ealing,Private room,70,2,2,0.07,6,62
2,Islington,Private room,45,1,14,0.14,2,364


In [18]:
#verifico los valores maximos de las columnas del dataframe
np.max(london_filt)

neighbourhood                     Westminster
room_type                         Shared room
price                                     299
minimum_nights                             13
number_of_reviews                         536
reviews_per_month                       15.56
calculated_host_listings_count           1034
availability_365                          365
dtype: object

In [19]:
#Genero los dumies para las Features categoricas
# dummies para la feature "neighbourhood"
neighs_dummie = pd.get_dummies(london_filt.neighbourhood)
neighs_dummie.head(3)

,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#lo mismo de antes pero para "romm type"
room_dummie = pd.get_dummies(london_filt.room_type)
room_dummie.head(3)

,Entire home/apt,Private room,Shared room
0,0,1,0
1,0,1,0
2,0,1,0


In [21]:
#uno los dumies a mi dataframe
london_filt = london_filt.join([neighs_dummie, room_dummie])
london_filt.head(3)

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Barking and Dagenham,Barnet,...,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster,Entire home/apt,Private room,Shared room
0,Haringey,Private room,35,1,131,1.71,4,262,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Ealing,Private room,70,2,2,0.07,6,62,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Islington,Private room,45,1,14,0.14,2,364,0,0,...,0,0,0,0,0,0,0,0,1,0


## Empiezo con los modelos

In [22]:
#creo mi variable dependiente "etiqueta-label"
y = np.array(london_filt[["price"]])

In [24]:
#creo la variable independiente elimandno las features que no quiero
#como ya tengo los dumies elimino "room type" y "neighbourhood". y "price" por ser la variable dependiente
x = london_filt.drop(['price', 'neighbourhood','room_type'], axis=1)
np.shape(x)

(51827, 41)

### Divido el dataset en Train y Test

In [25]:
#me interea que mi train sea solo el 10% del data set
# El random state funciona para que cada vezx que ejecute se mantengan los datos de train y test
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.90, random_state=42)

In [28]:
xtrain.shape

(5182, 41)

In [29]:
xtest.shape

(46645, 41)

In [30]:
#normalizo y escalo mis datos de train. auto scaling train- set (mean = 0, std = 1)
scaler = preprocessing.StandardScaler().fit(xtrain)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [31]:
# auto scalo mis muestras de train utilizando el scaler fiteado con el xtrain
xtrain_scal = scaler.transform(xtrain) 

In [32]:
# auto scalo mis muestras de test utilizando el scaler fiteado con el xtrain
xtest_scal = scaler.transform(xtest)  

### Creo un modelo de regresion lineal

In [34]:
#creo un modelo generico de regresion lineal
lr = LinearRegression()

In [35]:
#Ajusto el modelo de regresion lineal utilizando el set de train, tanto las features X como las samples Y
#lo entreno
lr.fit(xtrain_scal, ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [36]:
# Obtengo las predicciones que realiza mi modelo con las muestras de test, sin mostrarle las labels (Ytest)
# las predicciones las guardo en el vector "ypred"
ypred = lr.predict(xtest_scal)

In [37]:
# calculo el error de mi modelo con las muestras de train = error de train
#lo comparo con el ytest ya que son los valores verdaderos y veo que tan errado estuvo
np.sqrt(mean_squared_error(ytest, ypred))

39.306858993183056

In [38]:
#calculo el error cuadratico medio de mi modelo
mean_squared_error(ytest, ypred)

1545.0291639099755

In [39]:
from sklearn.metrics import mean_absolute_error

In [40]:
mean_absolute_error(ytest, ypred)

28.10456873227518

### Modelo SVR

In [41]:
# DEfino el estimador para el svr
est = SVR(max_iter=25000)
# Determino los parametros del modelo
parameters = {'C' : [1,100],
              'kernel':['rbf'],
             'gamma':[0.1, 0.5]}
n_folds=5
# Definimos nuevamente el objeto GS con los parametros previamente definidos
gs = GridSearchCV(est, param_grid=parameters,refit=True,
                  cv=n_folds, scoring="neg_mean_squared_error",
                  verbose=3, n_jobs=3)

In [42]:
# Entreno al modelo con el xtrain escalado y el ytrain antes definidos
gs.fit(xtrain_scal, ytrain)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:   32.5s finished
C:\Users\alcur\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=25000, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=3,
             param_grid={'C': [1, 100], 'gamma': [0.1, 0.5], 'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=3)

In [43]:
#muestro los mejores resultados del GS
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=25000, shrinking=True, tol=0.001, verbose=False) 

{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'} 

-1680.886555843345 



In [44]:
# Realizo la prediccion con el mejor estimador del Gs
#guardo la prediccion en un nuevo vector
svr_prediction = gs.best_estimator_.predict(xtest_scal)

In [45]:
#calculo el r2 y los errores del modelo
svr_r2 = r2_score(y_true=ytest, y_pred=svr_prediction)
svr_mse = mean_squared_error(y_true=ytest, y_pred=svr_prediction)
svr_mae = mean_absolute_error(y_true=ytest, y_pred=svr_prediction)

In [46]:
#printeo los resultados obtenidos del modelo
print(f'R2 score: {svr_r2:.6f}')
print(f'MAE: {svr_mae:.6f}')
print(f'MSE: {svr_mse:.6f}')

R2 score: 0.479678
MAE: 27.193622
MSE: 1629.680183


### Modelo KNN

In [48]:
from sklearn.neighbors import KNeighborsRegressor as KN

In [49]:
# Creo el estimador para el KNN
est_knn= KN()
# Determino los paramtros (son los que dio agustin)
parameters_k = np.arange(20,31,5)
parameters_knn = [{'n_neighbors': parameters_k}]
n_folds=5
# Definimos nuevamente el objeto GS con los parametros previamente definidos
gs = GridSearchCV(est_knn,param_grid=parameters_knn,refit=True,
                 cv=n_folds, scoring="neg_mean_squared_error",
                verbose=3, n_jobs=3)

In [50]:
# Entreno al modelo con el xtrain escalado y el ytrain antes definidos
gs.fit(xtrain_scal, ytrain)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  15 out of  15 | elapsed:    4.6s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=3,
             param_grid=[{'n_neighbors': array([20, 25, 30])}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=3)

In [51]:
#muestro los mejores resultados del GS
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                    weights='uniform') 

{'n_neighbors': 20} 

-1633.0440172967944 



In [52]:
# Realizo la prediccion con el mejor estimador del Gs
#guardo la prediccion en un nuevo vector
KNN_prediction = gs.best_estimator_.predict(xtest_scal)

In [53]:
#calculo el r2 y los errores del modelo
KNN_r2 = r2_score(y_true=ytest, y_pred=KNN_prediction)
KNN_mse = mean_squared_error(y_true=ytest, y_pred=KNN_prediction)
KNN_mae = mean_absolute_error(y_true=ytest, y_pred=KNN_prediction)

In [54]:
#printeo los resultados obtenidos del modelo
print(f'R2 score: {KNN_r2:.6f}')
print(f'MAE: {KNN_mae:.6f}')
print(f'MSE: {KNN_mse:.6f}')

R2 score: 0.493432
MAE: 28.067338
MSE: 1586.600613


### Modelo Random Forest regressor

In [57]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [63]:
# Creo el estimador para el KNN
est_RF= RandomForestRegressor()
# Determino los paramtros (son los que dio agustin)
parameters_RF = {'n_estimators': [100, 200, 300, 1000]}
n_folds=5
# Definimos nuevamente el objeto GS con los parametros previamente definidos
gs = GridSearchCV(est_RF,param_grid=parameters_RF,refit=True,
                 cv=n_folds, scoring="neg_mean_squared_error",
                verbose=3, n_jobs=3)

In [64]:
# Entreno al modelo con el xtrain escalado y el ytrain antes definidos
gs.fit(xtrain_scal, ytrain)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:  2.2min finished
C:\Users\alcur\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [65]:
#muestro los mejores resultados del GS
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False) 

{'n_estimators': 1000} 

-1599.0633429422596 



In [67]:
# Realizo la prediccion con el mejor estimador del Gs
#guardo la prediccion en un nuevo vector
RF_prediction = gs.best_estimator_.predict(xtest_scal)

In [68]:
#calculo el r2 y los errores del modelo
RF_r2 = r2_score(y_true=ytest, y_pred=RF_prediction)
RF_mse = mean_squared_error(y_true=ytest, y_pred=RF_prediction)
RF_mae = mean_absolute_error(y_true=ytest, y_pred=RF_prediction)

In [69]:
#printeo los resultados obtenidos del modelo
print(f'R2 score: {RF_r2:.6f}')
print(f'MAE: {RF_mae:.6f}')
print(f'MSE: {RF_mse:.6f}')

R2 score: 0.488912
MAE: 28.127191
MSE: 1600.758364


In [85]:
#CReo un data frame de resultados
results_df = pd.DataFrame(columns=['Model','R2','MSE','MAE'])

In [86]:
#guardo los resultados
results_df = results_df.append({'Model':'SVR',
                                'R2':svr_r2,
                                'MSE':svr_mse,
                                'MAE':svr_mae},ignore_index=True)
results_df = results_df.append({'Model':'KNN',
                                'R2':KNN_r2,
                                'MSE':KNN_mse,
                                'MAE':KNN_mae},ignore_index=True)
results_df = results_df.append({'Model':'RF',
                                'R2':RF_r2,
                                'MSE':RF_mse,
                                'MAE':RF_mae},ignore_index=True)

In [87]:
results_df

,Model,R2,MSE,MAE
0,SVR,0.479678,1629.680183,27.193622
1,KNN,0.493432,1586.600613,28.067338
2,RF,0.488912,1600.758364,28.127191


#### Como podemos observar el mejor modelo resulto el KNN